# Imports and Data Loading

In [1]:
%reload_ext autoreload
%matplotlib widget 
import h5py
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import pandas as pd
import os 
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime

/var/folders/ry/h8cjd1kx3r52swkrn4nnzzhh0000gn/T/ipykernel_17033/2441700993.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Load data and display the sessions 
animal="WI" # "FN" or "WI" 
data_file = f"/Users/withercp/Documents/dev/brainhack/data/TH_task_{animal}_singleprobe_500ms.h5"
assert os.path.exists(data_file), "File not found: {data_file}".format(data_file=data_file)

with h5py.File(data_file, 'r') as f:
    sessions = list(f.keys())

In [5]:
# Load data for a particular session and view the format of the data
i = 0
session = sessions[i]
# convert mm-dd-yyyy to mm/dd/yyyy
date_str = session.replace('-', '/')

with h5py.File(data_file, 'r') as f:
    print(f[sessions[i]].keys())
    epoc_np=np.array(f[sessions[i]]['epoch'])
    rem_rec_np=np.array(f[sessions[i]]['rem_rec'])
    trial_np=np.array(f[sessions[i]]['trial'])
    binned_spike_np = np.array(f[sessions[i]]['binned_spike'])
    meta = np.array(f[sessions[i]]['meta'])

rem_rec_bin = np.array([1 if x == b'Remote' else 0 for x in rem_rec_np])

block_num = 0
block_nums = np.zeros(len(rem_rec_bin), dtype=int)  # Initialize array with zeros

for i in range(1, len(rem_rec_bin)):
    if rem_rec_bin[i] != rem_rec_bin[i-1]:  # If trial type changes
        block_num += 1  # Increment block number
    block_nums[i] = block_num  # Assign block number to current trial

unique_blocks = np.unique(block_nums)

<KeysViewHDF5 ['binned_spike', 'epoch', 'meta', 'rem_rec', 'trial']>


In [ ]:
# Get correct and incorrect trials per screen (I still need approval to use this data)

'''
df = pd.read_csv("Y:/LAMBDA/Sub-Project/SLUR(..)/data/FN/FN_TT_ALL_refined.csv")
df = df[df['Date'] == date_str]
print(df.head())

screen_correct = np.zeros((4, df['TrialNum'].max()))
screen_correct[0, :] = df["Correct"][::4]
screen_correct[1, :] = df["Correct"][1::4]
screen_correct[2, :] = df["Correct"][2::4]
screen_correct[3, :] = df["Correct"][3::4]
screen_correct_drift = screen_correct[:, :-1]
'''

In [6]:
# Only keep rows where in at least one bin, the neuron fired at least 500 times 
# (i.e. the neuron is not active in that bin)

rows_to_keep = [] 
for i in unique_blocks:
    for j in range(binned_spike_np.shape[0]):
        if np.sum(binned_spike_np[j, block_nums == i]) >= 500:
            rows_to_keep.append(j)

rows_to_keep = np.unique(rows_to_keep)

binned_spike_np = binned_spike_np[rows_to_keep, :]

# Normalize binned spike data 
binned_spike_np_z = scipy.stats.zscore(binned_spike_np, axis=1)

# Decoding

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Simple logistic regression classifier to predict behavior (classification) from low dimensional data (data)

def logistic_regression_classifier(data, classification):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data, classification, test_size=0.2, random_state=42)

    # Fit the model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


# PCA

In [ ]:
# PCA on all labels

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def PCA_binned_spike_np(X, y):
    pca = PCA(n_components=3)
    pca.fit(np.transpose(X))
    embedding = pca.transform(np.transpose(X))

    # Plot PCA embedding and color by value in epoc_np
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(embedding[:,0], embedding[:,1], embedding[:,2], c=y, cmap = 'coolwarm')
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    plt.show()
    return embedding

PCA_binned_spike_np(binned_spike_np, trial_np)

# t-SNE

In [ ]:
from sklearn.manifold import TSNE

def TSNE_binned_spike_np(X, y):
    # t-SNE
    tsne = TSNE(n_components = 128, perplexity=100)
    embedding = tsne.fit_transform(np.transpose(X))

    # Plot the first three components in 3d and make the plot interactive
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    scatter_plot = ax.scatter(embedding[:,0], embedding[:,1], embedding[:,2], c = y)
    ax.set_xlabel('t-SNE1')
    ax.set_ylabel('t-SNE2')
    ax.set_zlabel('t-SNE3')
    plt.show()

    return embedding

TSNE_binned_spike_np(binned_spike_np, trial_np)

In [ ]:
# Use optuna (bayesian optimization) to find the best hyperparameters for t-SNE to maximize classification accuracy

import optuna
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = binned_spike_np # neural data
y = rem_rec_np # classification labels

# Simple logistic regression classifier to predict behavior (classification) from low dimensional data (data)
def logistic_regression_classifier(data, classification):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data, classification, test_size=0.2, random_state=42)

    # Fit the model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Define the objective function for Optuna
def objective(trial):
    # Define the t-SNE object with the hyperparameters to optimize
    tsne = TSNE(
        n_components=3,
        perplexity=trial.suggest_int('perplexity', 5, len(y)-1),
        learning_rate=trial.suggest_float('learning_rate', 10, 100),
        n_iter=trial.suggest_int('n_iter', 250, 1000),
        random_state=42
    )

    # Transform the data with t-SNE
    tsne_embedding = tsne.fit_transform(np.transpose(X))
    accuracy = logistic_regression_classifier(tsne_embedding, y)
    return accuracy

# Define the study object for Optuna
study = optuna.create_study(direction='maximize')

# Optimize the hyperparameters with Optuna
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and score
print("Best parameters: ", study.best_params)
print("Best score: ", study.best_value)

# Plot the t-SNE embedding with the best hyperparameters
tsne = TSNE(
    n_components=3,
    perplexity=study.best_params['perplexity'],
    learning_rate=study.best_params['learning_rate'],
    n_iter=study.best_params['n_iter'],
    random_state=42
)
tsne_embedding = tsne.fit_transform(np.transpose(X))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(tsne_embedding[:,0], tsne_embedding[:,1], tsne_embedding[:,2], c=y, cmap='jet')
ax.set_xlabel('t-SNE1')
ax.set_ylabel('t-SNE2')
ax.set_zlabel('t-SNE3')
ax.set_title('t-SNE on Block Segment (Firing Rate)')

clf = LogisticRegression(random_state=0)
score = cross_val_score(clf, tsne_embedding, y, cv=5).mean()
print("Classification accuracy for remote/recent:", score)

# UMAP

In [ ]:
import umap
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from sklearn.preprocessing import StandardScaler

def UMAP_neural(X = binned_spike_np, y = trial_np, params = {'n_dims':3, 'metric': 'euclidean', 'n_neighbors':200, 'min_dist':0.2, 'init': 'spectral'}): 
    reducer = umap.UMAP(n_components=128, n_neighbors=params['n_neighbors'], min_dist=params['min_dist'], metric=params['metric'], init='spectral', random_state=42)
    scaled_block_firing_rate = StandardScaler().fit_transform(np.transpose(X))
    embedding = reducer.fit_transform(scaled_block_firing_rate)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    scatter_plot = ax.scatter(embedding[:,0], embedding[:,1], embedding[:,2], c = epoc_np, cmap='coolwarm')
    ax.set_xlabel('UMAP1')
    ax.set_ylabel('UMAP2')
    ax.set_zlabel('UMAP3')
    #fig.colorbar(scatter_plot, ax=ax)
    plt.show()
    return embedding

UMAP_neural()

In [9]:
# Parallel processing for UMAP hyperparameter tuning

from joblib import Parallel, delayed
import os
import warnings

# Get the number of cores available
num_cores = os.cpu_count()
print(f"Number of cores available: {num_cores}")

def UMAP_neural(X = binned_spike_np, y = trial_np, params = {'n_dims':3, 'metric': 'euclidean', 'n_neighbors':200, 'min_dist':0.2, 'init': 'spectral'}):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        print(f"Worker {os.getpid()} is tuning hyperparameters: {params}")
        reducer = umap.UMAP(n_components=128, n_neighbors=params['n_neighbors'], min_dist=params['min_dist'], metric=params['metric'], init='spectral', random_state=42)
        
        embedding = reducer.fit_transform(scaled_block_firing_rate)
        return params, embedding

# Run UMAP with different n_neighbors and min_dist values in parallel
n_neighbors = [200, 400, 600, 800, 1000, 1200]
min_dist = [0.2, 0.4, 0.6, 0.8, 1.0]

scaled_block_firing_rate = StandardScaler().fit_transform(np.transpose(binned_spike_np))

# Running 10 cores at a time across 30 hyperparameter combinations takes around 13 minutes on my computer
# I might try reducing num_cores to 5 or 6 to reduce overhead 
r1 = Parallel(n_jobs=num_cores//2)(delayed(UMAP_neural) (X = scaled_block_firing_rate, y = trial_np, params = {'n_dims':3, 'metric': 'euclidean', 'n_neighbors':i, 'min_dist':j, 'init': 'spectral'}) for i in n_neighbors for j in min_dist)


Number of cores available: 10
Worker 17052 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 200, 'min_dist': 0.6, 'init': 'spectral'}Worker 17051 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 200, 'min_dist': 1.0, 'init': 'spectral'}

Worker 17055 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 200, 'min_dist': 0.8, 'init': 'spectral'}
Worker 17053 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 200, 'min_dist': 0.2, 'init': 'spectral'}
Worker 17054 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 200, 'min_dist': 0.4, 'init': 'spectral'}


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Worker 17051 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 400, 'min_dist': 0.2, 'init': 'spectral'}
Worker 17052 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 400, 'min_dist': 0.4, 'init': 'spectral'}
Worker 17053 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 400, 'min_dist': 0.6, 'init': 'spectral'}
Worker 17054 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 400, 'min_dist': 0.8, 'init': 'spectral'}
Worker 17055 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 400, 'min_dist': 1.0, 'init': 'spectral'}
Worker 17053 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 600, 'min_dist': 0.2, 'init': 'spectral'}
Worker 17055 is tuning hyperparameters: {'n_dims': 3, 'metric': 'euclidean', 'n_neighbors': 600, 'min_dist': 0.4, 'init': 'spectral'}
Worker 17052 is tuning hyperparameters: {'n_dims': 3, 'metric'

In [17]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import itertools

def plot_embedding(embedding):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    scatter_plot = ax.scatter(embedding[:, 0], embedding[:, 1], embedding[:, 2], c=trial_np, cmap='coolwarm')
    cbar = plt.colorbar(
        scatter_plot,

    )
    cbar.set_label("Trial #")
    ax.set_xlabel('UMAP1')
    ax.set_ylabel('UMAP2')
    ax.set_zlabel('UMAP3')
    plt.show()

# Slider setup
slider_neighbors = widgets.IntSlider(min=200, max=2000, step=200, value=200)
slider_dist = widgets.FloatSlider(min=0.2, max=1.0, step=0.2, value=0.2)

# Display the sliders
display(slider_neighbors)
display(slider_dist)

n_neighbors = [200, 400, 600, 800, 1000, 1200]
min_dist = [0.2, 0.4, 0.6, 0.8, 1.0]

pairs = list(itertools.product(n_neighbors, min_dist))

@widgets.interact(n_neighbors=slider_neighbors, min_dist=slider_dist)
def update(n_neighbors, min_dist):
    for embedding in r1: 
        if embedding[0]['n_neighbors'] == n_neighbors and embedding[0]['min_dist'] == min_dist:
            plot_embedding(embedding[1])


IntSlider(value=200, max=2000, min=200, step=200)

FloatSlider(value=0.2, max=1.0, min=0.2, step=0.2)

interactive(children=(IntSlider(value=200, description='n_neighbors', max=2000, min=200, step=200), FloatSlide…

In [ ]:
# Use optuna (bayesian optimization) to find the best hyperparameters for UMAP to maximize classification accuracy 

import optuna
from umap import UMAP
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

X = binned_spike_np
y = rem_rec_np

def logistic_regression_classifier(data, classification):
    X_train, X_test, y_train, y_test = train_test_split(data, classification, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


def objective(trial):
    umap = UMAP(
        n_components=3,
        n_neighbors=trial.suggest_int('n_neighbors', 100, len(y)-1),
        min_dist=trial.suggest_float('min_dist', 0.01, 1),
        spread=trial.suggest_float('spread', 1.0, 1.5),
        metric=trial.suggest_categorical('metric', ['cosine', 'euclidean']), 
    )
    umap_embedding = umap.fit_transform(np.transpose(X))
    #accuracy = logistic_regression_classifier(umap_embedding, y)
    accuracy = logistic_regression_classifier(umap_embedding)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print("Best parameters: ", study.best_params)
print("Best score: ", study.best_value)

umap = UMAP(
    n_components=3,
    n_neighbors=study.best_params['n_neighbors'],
    min_dist=study.best_params['min_dist'],
    spread=study.best_params['spread']
)
umap_embedding = umap.fit_transform(np.transpose(X))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(umap_embedding[:,0], umap_embedding[:,1], umap_embedding[:,2], c=y, cmap='jet')
ax.set_xlabel('UMAP1')
ax.set_ylabel('UMAP2')
ax.set_zlabel('UMAP3')
ax.set_title('UMAP (Firing Rate)')

clf = LogisticRegression(random_state=0)
score = cross_val_score(clf, umap_embedding, y, cv=5).mean()
print("Classification accuracy for remote/recent:", score)

# CEBRA

In [ ]:
# Default cebra model (no behavioral labels) creates a latent space to minimize distance between 
# points that share similar times 

from cebra import CEBRA
cebra_model = CEBRA(
    model_architecture = "offset10-model",
    batch_size = 1024,
    temperature_mode="auto",
    learning_rate = 0.001,
    max_iterations = 10,
    time_offsets = 10,
    output_dimension = 3,
    device = "cuda_if_available",
    distance = "cosine", 
    verbose = True
)
cebra_model.fit(np.transpose(binned_spike_np), trial_np)
cebra_time = cebra_model.transform(np.transpose(binned_spike_np))

# Plot cebra_time in 3 dimensions
y = trial_np
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(cebra_time[:,0], cebra_time[:,1], cebra_time[:,2], c=y, cmap='jet')
ax.set_xlabel('CEBRA1')
ax.set_ylabel('CEBRA2')
ax.set_zlabel('CEBRA3')
ax.set_title('CEBRA (Firing Rate)')
plt.show()

In [ ]:
# Now we will try to use the behavioral labels to create a latent space that minimizes distance between
# points that share similar behaviors. BUT, we shuffle the time labels to see if there really exists 
# a low dimensional structure that is specific to time 

trial_np_shuffled = np.random.permutation(trial_np)
cebra_model.fit(np.transpose(binned_spike_np), trial_np_shuffled)
cebra_time_shuffled = cebra_model.transform(np.transpose(binned_spike_np))

# Plot cebra_time in 3 dimensions
y = trial_np_shuffled
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(cebra_time_shuffled[:,0], cebra_time_shuffled[:,1], cebra_time_shuffled[:,2], c=y, cmap='jet')
ax.set_xlabel('CEBRA1')
ax.set_ylabel('CEBRA2')
ax.set_zlabel('CEBRA3')
ax.set_title('CEBRA (Firing Rate)')
plt.show()

In [ ]:
# Using the smallest model (offset1-model) to see if we can get a better latent space faster

import cebra
from sklearn.model_selection import train_test_split

models = ['offset10-model','offset5-model','offset1-model','offset40-model-4x-subsample', 'resample5-model', 'offset36-model', 'offset36-model-dropout']
models = ['offset1-model']
embeddings = []
X = binned_spike_np
y = np.stack((rem_rec_bin, epoc_np, trial_np))

n = len(trial_np)  # Number of trials
num_samples = int(0.2 * n)  # 20% of the number of trials

# Generate random numbers
random_numbers = np.random.randint(0, n, num_samples)

# Split the data into training and testing sets
X_train = X[random_numbers, :]

# 1. Define the parameters, either variable or fixed
for model in models:
    cebra_model = cebra.CEBRA(
        model_architecture = model,
        output_dimension = 4,
        learning_rate = 3e-4,
        time_offsets = 10,
        max_iterations = 10000,
        temperature_mode = "auto",
        distance = "cosine", 
        batch_size = 2048,
        conditional="time_delta",
        num_hidden_units=128,
        verbose = True)
    cebra_model.fit(np.transpose(X_train), np.transpose(y_train))
    cebra_time = cebra_model.transform(np.transpose(X_train))
    embeddings.append(cebra_time)
    
    save_file = f"saved_models/{model}.pt"
    cebra_model.save(save_file)

In [ ]:
i = 0
y = trial_np[(epoc_np == 1) | (epoc_np == 2) | (epoc_np == 3) | (epoc_np == 4) | (epoc_np == 5)]
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(embeddings[i][:,0], embeddings[i][:,1], embeddings[i][:,2], c=y, cmap='jet')
ax.set_xlabel('CEBRA1')
ax.set_ylabel('CEBRA2')
ax.set_zlabel('CEBRA3')
ax.set_title('CEBRA (Firing Rate)')
plt.show()

In [ ]:
# Display the loss 

fig = plt.figure()
ax = plt.subplot(111)
ax.plot(cebra_model.state_dict_['loss'])
fig.show()